# Space

In [ ]:
import os
import logging
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
KEY = 'WorkSpace'
WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY
# print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
import sys
from proj_space import SPACE
sys.path.append(SPACE['CODE_FN'])
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

from datasets import disable_caching
disable_caching()

SPACE['MODEL_ENDPOINT'] = 'vTest'



In [ ]:
import os
import logging
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML
pd.set_option('display.max_columns', None)
KEY = 'WorkSpace'
WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY
# print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
import sys
from proj_space import SPACE
sys.path.append(SPACE['CODE_FN'])
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='[%(levelname)s:%(asctime)s:(%(filename)s@%(lineno)d %(name)s)]: %(message)s')

from datasets import disable_caching
disable_caching()

SPACE['MODEL_ENDPOINT'] = 'vTest'



# Part 1: AIData

In [ ]:
from recfldtkn.aidata_base.aidata import AIData

DATA_AIDATA = SPACE['DATA_AIDATA']
OneAIDataName = 'CgmLhm_Bf24Af2Af2t8H_5Min_3Cohort_EventFlt_Sample'

aidata = AIData.load_aidata(DATA_AIDATA, OneAIDataName, SPACE)
aidata

In [ ]:
OneEntryArgs = {
    # ----------------- Task Part -----------------
    'Task_Part': {

        'Tagging': {
            # 'TagName_to_TaggingMethod': {
            #     # TagName: TaggingMethod {Rules: [(x,x,x)], Op: and or}
            # },
            # 'ColumnsAddToDsCase': [],
            'TagFilter': True, # <--- still need to add Fitlter Tag, as we need to do the RandomDownSample.
            'TagSplit': False, # <--- do not need to add Split Tag anymore, as we already have. 
        },

        'Filtering': {
            # 'FilterTagging': None,
            'FilterTagging': {
                "Rules": [
                    ['RandDownSample', '<=', 0.5],
                    ['co.Bf24H_Food_recnum:recnum', '>=', 1], 
                    ], 
                'Op': 'and',
            }
        }, 
        
        'Splitting': {
            # 'SplitTagging': { # <----- for the Tagging part.
            #     'RANDOM_SEED': 32,
            #     'out_ratio': 0.1,
            #     'test_ratio': 'tail0.1',
            #     'valid_ratio': 0.1
            # },
            'TrainEvals': {
                'TrainSetName': 'In-Train', 
                'EvalSetNames': ['In-Test', 'In-Valid', 'Out']
            },
        }
    },

    # ----------------- Input Part -----------------
    'Input_Part': {
        'EntryInputMethod': 'Mto1Period_MultiTknInStep',
        'CF_list': [
            'cf.TargetCGM_Bf24H',
            'cf.TargetCGM_Af2H',

            'cf.ActivitySparse_Bf24H',
            'cf.ActivitySparse_Af2H',

            # 'cf.TimeSparse_Bf24H', 
            # 'cf.TimeSparse_Af2H',


            'cf.DietSparse_Bf24H',
            'cf.DietSparse_Af2H',
        ],
        'TargetField': 'TargetCGM',
        # 'TimeField':   'Time',
        'EventFields': [
            'Activity',
            'Diet',
        ],
        'BeforePeriods': ['Bf24H'],
        'AfterPeriods': ['Af2H'],
        'InferenceMode': False, # 'WithFutureEvent' #  # 'NoFutureEvent', 'WithFutureEvent', 
    }, 

    # ----------------- Output Part -----------------
    'Output_Part': {
        'EntryOutputMethod': 'EventPrediction',
        # 'MaskingRate': 0,
        'Task_Label': 'Diet',
        # other parameters toward X and Y value
        'agg_function':None,
        'label_process': None, 
        'use_gaussian_blur': False,
    },
}


aidata.update_NameToData_with_OneEntryArgs(OneEntryArgs)

In [ ]:
# aidata.Name_to_DsAIData
split_name = [i for i in  aidata.Name_to_Data][0]
Name_to_Data = aidata.Name_to_Data# [split_name]
Data = Name_to_Data[split_name]
df_case = Data['df_case']
df_case.head()

In [ ]:
ds_tfm = Data['ds_tfm']
ds_tfm

In [ ]:
Data['df_case'].columns

In [ ]:
batch_size = 4
batch = ds_tfm[:batch_size]
batch

In [ ]:
batch['input_ids'].shape

In [ ]:
batch['labels'][0]# .sum(axis = 1)

In [ ]:
batch

In [ ]:
# # aidata.CF_to_CFvocab

# aidata.OneEntryArgs

In [ ]:
# import numpy as np
# import evaluate  # New Hugging Face library for evaluation metrics

# # Load accuracy metric
# metric = evaluate.load("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = np.argmax(logits, axis=-1)  # Get highest probability token

#     # Flatten for sequence-based accuracy
#     preds = preds.flatten()
#     labels = labels.flatten()

#     return metric.compute(predictions=preds, references=labels)


In [ ]:
# from datasets import Dataset
# import torch
# from transformers import RobertaTokenizer

# batch_size = 4
# # Hugging Face Dataset expects input as a dictionary
# data = ds_tfm[:batch_size]

# # Create Hugging Face Dataset
# dataset = Dataset.from_dict(data)

# # Load tokenizer
# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# pad_token_id = tokenizer.pad_token_id
# max_length = 8

# # Define padding and truncation function
# def preprocess_function(examples):
#     # Truncate and pad input_ids
#     padded_inputs = []
#     attention_masks = []
#     for input_id in examples["input_ids"]:
#         # Truncate or pad to max_length
#         if len(input_id) > max_length:
#             input_id = input_id[:max_length]
#         else:
#             input_id = input_id + [pad_token_id] * (max_length - len(input_id))
        
#         # Generate attention mask
#         attention_mask = [1 if token != pad_token_id else 0 for token in input_id]
        
#         padded_inputs.append(input_id)
#         attention_masks.append(attention_mask)
    
#     examples["input_ids"] = padded_inputs
#     examples["attention_mask"] = attention_masks
#     return examples

# # Apply the preprocessing to the dataset
# processed_dataset = dataset.map(preprocess_function, batched=True)

# processed_dataset.set_format(
#     type="torch", 
#     columns=["input_ids", "attention_mask", "labels"]
# )


# # Fetch a batch (batch_size = 4)
# batch_size = 4
# batch = processed_dataset[:batch_size]
# print(batch)


In [ ]:
# batch['input_ids'] .shape

In [ ]:
# input_ids = processed_dataset['input_ids']
# input_ids

In [ ]:
# input_ids

In [ ]:
# input_ids[2, :] # 313 = 288 (24h) +  1 (obspoint) + 24 (2h)

In [ ]:
# labels = batch['labels']
# labels.shape

In [ ]:
# batch

# Part 2: Model Init

## CgmEventConfig

In [ ]:
ModelArgs = {
    'model_type': 'cgm_encoder',
}

In [ ]:
import transformers
import logging
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_CAUSAL_LM_MAPPING,
    AutoConfig,
    HfArgumentParser,
    TrainingArguments,
)
MODEL_CONFIG_CLASSES = list(MODEL_FOR_CAUSAL_LM_MAPPING.keys())
# MODEL_CONFIG_CLASSES
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)
# MODEL_TYPES

############# this is the NN development that showing our novelty #############
from nn.cgmevent.configuration_fieldencoder import FieldEncoderConfig
# from nn.cgmevent.modeling_fieldencoder_cgm import FieldEncoderModel
from nn.cgmevent.modeling_fieldencoder import FieldEncoderForStepClassification
#################################################################

In [ ]:
# %load_ext autoreload
# %autoreload 2

In [ ]:
# ----- within the method of init_model.
CF_to_CFvocab = aidata.CF_to_CFvocab
CF = list(CF_to_CFvocab.keys())[0]
CFvocab = CF_to_CFvocab[CF]
tkn2tid = CFvocab['input_ids']['tkn2tid']

config_kwargs = {
    # "cache_dir": model_args.cache_dir,
    # "revision": model_args.model_revision,
    # "token": model_args.token,
    # "trust_remote_code": model_args.trust_remote_code,
    ###########
    'vocab_size': len(tkn2tid),
    'bos_token_id': tkn2tid['[BOS]'],
    'eos_token_id': tkn2tid['[EOS]'],
    'pad_token_id':  0,
    ###########
}

ModelArgs.update(config_kwargs)

# pprint(ModelArgs)
config = FieldEncoderConfig(**ModelArgs)
pprint(config)

In [ ]:
model = FieldEncoderForStepClassification(config) 
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

print(model)

# Part 3: Forward

In [ ]:
import numpy as np 
import torch 


batch2dp = 8

batch = ds_tfm.select(range(batch2dp))[:batch2dp]
inputs = batch 

input_batch = {'input_ids': torch.LongTensor(inputs['input_ids']),
               'labels': torch.LongTensor(inputs['labels'])}
for k, v in input_batch.items():
    print(k, v.shape)   
    
inputs = batch

In [ ]:
input_batch

In [ ]:
transformer_outputs = model.forward(**input_batch)
transformer_outputs

In [ ]:
[i for i in transformer_outputs]

In [ ]:
transformer_outputs['logits'].shape

In [ ]:
output = model(**input_batch)
output

In [ ]:
dict(output).keys()

In [ ]:
input_batch